In [7]:
import urllib.request
import os
import glob
import shutil

In [35]:
directory = os.path.dirname(os.getcwd())
directory
path= directory+'/Part_2/Log'
# path = directory+'/final/data/'
path

'/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/Log'

### Downloading the files in zip

In [36]:
year ='2013'
from urllib.request import urlopen
from tempfile import NamedTemporaryFile
from shutil import unpack_archive
zipurl = 'http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2003/Qtr1/log20030201.zip'
zipresp = urlopen(zipurl)
tfile = NamedTemporaryFile()
tfile.write(zipresp.read())
tfile.seek(0)
unpack_archive(tfile.name,path, format = 'zip')
zipresp.close()
tfile.close()

In [ ]:
file_name = cur_path+'Data/'\
    +'/' + dict['country'] + '/' + dict['city'] + '/' + dict['name'] \
                + '.json'
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    shutil.move(file,file_name) #Moving the file from TEMP folder to required destination